### Creating User Login

In [237]:
import pandas as pd
def signup():
    Name=input("Name:")
    Email=input("Email:")
    Phone=input("Phone:")
    Password=input("Password:")
    Con_password=input("Confirm Password:")
    if Con_password!=Password:
        print("Passwords don't match. Please Retry")
        signup()
    df=pd.read_csv('user_data.csv')
    df_2=df[df['email']==Email]
    if len(df_2)!=0:
        print("Email already exists try different email")
        signup()
        
    wr=open('user_data.csv','a')
    wr.write(Name+","+Email+","+Password+","+Phone+"\n")
    wr.close()    
    print("Now please log in")

In [238]:
def login():
    print("1 to Signup, 2 to Login")
    ch=int(input())
    if ch==1:
        signup()
    df=pd.read_csv('user_data.csv')
    Email=input("Email:")
    Pass=input("Pass:")
    
    df_2=df[df['email']==Email]
    #print(df_2)
    
    if len(df_2)==0:
        print("Email not found, try again")
        login()
    
    if str(df_2.iloc[0]['password'])==Pass:
        print("Welcome "+df.iloc[0]['Name'])
        surf(Email)
        
    else:
        print("Password Wrong, try again")
        login()    

### Obtaining the stories


In [330]:
from os import system, name 
def clear(): 
  
    # for windows 
    if name == 'nt': 
        _ = system('cls') 
  
    # for mac and linux(here, os.name is 'posix') 
    else: 
        _ = system('clear') 

In [239]:
import os
import pandas as pd
def get_stories():
    fldr='feeds'
    files=os.listdir(fldr)
    df_f=pd.DataFrame()
    
    for i in files:
        fl=fldr+'/'+i
        
        df=pd.read_csv(fl)
        if df_f is None:
            df_f=df
        else:
            df_f=pd.concat([df_f,df],axis=0)
    
    return df_f
        
    

In [240]:
from nltk.tokenize import sent_tokenize, word_tokenize 
def clean_sentence(sentence):
    #extracts=sent_tokenize(article)
    sentences=[]
  
    #print(extract)
    clean_sentence=sentence.replace("[^a-zA-Z0-9]"," ")   ## Removing special characters
    #print(clean_sentence)
    obtained=word_tokenize(clean_sentence) 
    #print(obtained)
    sentences.append(obtained)

    return sentence

In [241]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.cluster.util import cosine_distance
def get_similarity(sent_1,sent_2,stop_words):
  
  sent_1=[w.lower() for w in sent_1]
  sent_2=[w.lower() for w in sent_2]

  total=list(set(sent_1+sent_2)) ## Removing duplicate words in total set

  vec_1= [0] * len(total)
  vec_2= [0] * len(total)


  ## Count Vectorization of two sentences
  for w in sent_1:
    if w not in stop_words:
      vec_1[total.index(w)]+=1

  for w in sent_2:
    if w not in stop_words:
      vec_2[total.index(w)]+=1


  return 1-cosine_distance(vec_1,vec_2)


[nltk_data] Downloading package punkt to /home/abhijit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhijit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Handling stories

In [346]:
import ast
nltk.download('stopwords')
from nltk.corpus import stopwords
def handle_similar(df, Title,email):
    clear()
    sim=[]
    df_temp=df
    print("\n\n Here are some similar Stories \n\n")
    for i in range(len(df)):
        try:
            title=ast.literal_eval(df.iloc[i]['Titles'])[0]
            cleaned_1=clean_sentence(Title)
            cleaned_2=clean_sentence(title)
            stop_words = stopwords.words('english')
            s=get_similarity(cleaned_1,cleaned_2,stop_words)
            if s<0.95:
                sim.append(s)
            else:
                sim.append(0)
        except:
            sim.append(0)
        
    df_temp['Sim']=sim
        
    df_temp.sort_values(by=['Sim'], inplace=True,ascending=False)
    #print(df_temp['Sim'].head())    
        
    for i in range(5):
        try:
            res = ast.literal_eval(df_temp.iloc[i]['Titles']) 
            print(str(i+1)+"-> "+res[0])
            print(df_temp.iloc[i]['Source']+", Today")
        
            print('\n\n')
        except:
            continue
        
    print("1 for choosing proposed, 2 to go back")
    choice=int(input())
    if choice==2:
        surf(email)
    else:
        try:
            ind=int(input("Please Provide the index of the story"))
            webbrowser.open(df_temp.iloc[ind-1]['link'])
            time.sleep(3)

            file_u = open('user_records.json')
            users=json.load(file_u)

            if email not in users.keys():
                users[email]={}
                
                users[email]['news']=[df_temp[ind-1]['Source']+df_temp.iloc[ind-1]['Date']+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0]]
                lab=[z for z in ast.literal_eval(df_temp.iloc[ind-1]['labels'])]
                users[email]['labels']=lab
            else:
                users[email]['news'].append(df_temp.iloc[ind-1]['Source']+df_temp.iloc[ind-1]['Date']+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0])
                lab=[z for z in ast.literal_eval(df_temp.iloc[ind-1]['labels'])]
                for l in lab:
                    users[email]['labels'].append(l)
            with open("user_records.json", "w") as outfile: 
                json.dump(users, outfile)

            file_s = open('story_records.json')
            stories=json.load(file_s)
            key=df_temp.iloc[ind-1]['Source']+'$'+df_temp.iloc[ind-1]['Date']+'$'+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0]

            if key not in stories.keys():

                stories[key]=[email]
            else:
                stories[key].append(email)

            with open("story_records.json", "w") as outfile: 
                json.dump(stories, outfile) 
        except:
            a=1
        handle_similar(df,ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0],email)
        
        
    
    
        
        
        
        
        
    
    

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhijit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [345]:
def search(email,df):
    clear()
    search=input("search")
    df_temp=df
    sim=[]
    for i in range(len(df)):
        try:
            title=ast.literal_eval(df.iloc[i]['Titles'])[0]
            cleaned_1=clean_sentence(search)
            cleaned_2=clean_sentence(title)
            stop_words = stopwords.words('english')
            s=get_similarity(cleaned_1,cleaned_2,stop_words)
            if s<0.95:
                sim.append(s)
            else:
                sim.append(0)
        except:
            sim.append(0)
            
    df_temp['Sim']=sim
    df_temp.sort_values(by=['Sim'], inplace=True,ascending=False)
    #print(df_temp.head())
    print("\n\n Top 5 Results \n")
    for i in range(5):
        
            res = ast.literal_eval(df_temp.iloc[i]['Titles']) 
            print(str(i+1)+"-> "+res[0])
            print(df_temp.iloc[i]['Source']+" , "+df_temp.iloc[i]['Date'])
        
            print('\n\n')
        
    ind=int(input("Please Provide the index of the story"))
        #print(str(stories_checked.iloc[indices[ind-1]]['link']))
    
            #ind=int(input("Please Provide the index of the story"))
    webbrowser.open(df_temp.iloc[ind-1]['link'])
    time.sleep(3)
    
    try:
    
            file_u = open('user_records.json')
            users=json.load(file_u)

            if email not in users.keys():
                users[email]={}
                
                users[email]['news']=[df_temp[ind-1]['Source']+df_temp.iloc[ind-1]['Date']+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0]]
                lab=[z for z in ast.literal_eval(df_temp.iloc[ind-1]['labels'])]
                users[email]['labels']=lab
            else:
                users[email]['news'].append(df_temp.iloc[ind-1]['Source']+df_temp.iloc[ind-1]['Date']+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0])
                lab=[z for z in ast.literal_eval(df_temp.iloc[ind-1]['labels'])]
                for l in lab:
                    users[email]['labels'].append(l)
        
            with open("user_records.json", "w") as outfile: 
                json.dump(users, outfile)
    
            file_s = open('story_records.json')
            stories=json.load(file_s)
            key=df_temp.iloc[ind-1]['Source']+'$'+df_temp.iloc[ind-1]['Date']+'$'+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0]

            if key not in stories.keys():

                stories[key]=[email]
            else:
                stories[key].append(email)

            with open("story_records.json", "w") as outfile: 
                json.dump(stories, outfile) 
    except:
    
            a=1
    surf(email)
            
            
        
        
        
        
        

In [344]:
import json
def get_popular(email,df):
    clear()
    #print("A")
    df_temp=df
    file_u = open('user_records.json')
    users=json.load(file_u)
    file_s = open('story_records.json')
    stories=json.load(file_s)
    
    popularity=[]
    for i in range(len(df)):
        try:
            key=df.iloc[i]['Source']+'$'+df.iloc[i]['Date']+'$'+ast.literal_eval(df_temp.iloc[i]['Titles'])[0]
            if key in stories.keys():
                p=stories.get(key)
                if email in p:
                    popularity.append(0)
                else:
                    popularity.append(len(p))
            else:
                popularity.append(0)
        except:
            popularity.append(0)
    
    df_temp['Pop']=popularity 
    df_temp.sort_values(by=['Pop'], inplace=True,ascending=False)
    print("\n\n Top 10 Results \n")
    for i in range(10):
        try:
            res = ast.literal_eval(df_temp.iloc[i]['Titles']) 
            print(str(i+1)+"-> "+res[0])
            print(df_temp.iloc[i]['Source']+" , "+df_temp.iloc[i]['Date'])
        
            print('\n\n')
        except:
            continue
        
    ind=int(input("Please Provide the index of the story"))
        #print(str(stories_checked.iloc[indices[ind-1]]['link']))
    
            #ind=int(input("Please Provide the index of the story"))
    webbrowser.open(df_temp.iloc[ind-1]['link'])
    time.sleep(3)
    try:
            file_u = open('user_records.json')
            users=json.load(file_u)

            if email not in users.keys():
                users[email]={}
                
                users[email]['news']=[df_temp[ind-1]['Source']+df_temp.iloc[ind-1]['Date']+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0]]
                lab=[z for z in ast.literal_eval(df_temp.iloc[ind-1]['labels'])]
                users[email]['labels']=lab
            else:
                users[email]['news'].append(df_temp.iloc[ind-1]['Source']+df_temp.iloc[ind-1]['Date']+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0])
                lab=[z for z in ast.literal_eval(df_temp.iloc[ind-1]['labels'])]
                for l in lab:
                    users[email]['labels'].append(l)
                
            with open("user_records.json", "w") as outfile: 
                json.dump(users, outfile)

            file_s = open('story_records.json')
            stories=json.load(file_s)
            key=df_temp.iloc[ind-1]['Source']+'$'+df_temp.iloc[ind-1]['Date']+'$'+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0]

            if key not in stories.keys():

                stories[key]=[email]
            else:
                stories[key].append(email)

            with open("story_records.json", "w") as outfile: 
                json.dump(stories, outfile) 
    except:
            a=1
    surf(email)

    
    
            
        
    
        
    
    
    

### Content Based Filtering

In [343]:
import ast
def get_customized(email,df):
    clear()
    file_u = open('user_records.json')
    users=json.load(file_u)
    
    file_s = open('story_records.json')
    stories=json.load(file_s)
        
    df_temp=df
    
    sim=[]
    
    if email in users.keys():
        tag_usr=""
        temp=users.get(email)
        temp=temp.get('labels')
        i= len(temp)-1
        count=0
        while i>=0:
            if count==20:
                break
            tag_usr+=temp[i]+" "
            i-=1
            count+=1
        #print(tag_usr)
        for i in range(len(df)):
            try:
                key=df.iloc[i]['Source']+'$'+df.iloc[i]['Date']+'$'+ast.literal_eval(df_temp.iloc[i]['Titles'])[0]
                if key in stories.keys():
                    p=stories.get(key)
                    if email in p:
                        sim.append(0)
                    else:
                        tag_str=""
                        lab=ast.literal_eval(df_temp.iloc[i]["labels"])
                        for l in lab:
                            tag_str+=l+" "
                        cleaned_1=clean_sentence(tag_usr)
                        cleaned_2=clean_sentence(tag_str)
                        s=get_similarity(cleaned_1,cleaned_2)
                        sim.append(s)
                        
                        
                else:
                    sim.append(0)
            except:
                sim.append(0)
        df_temp['sim']=sim 
        df_temp.sort_values(by=['sim'], inplace=True,ascending=False)
        print("\n\n Top 10 Results \n")
        for i in range(10):
            try:
                res = ast.literal_eval(df_temp.iloc[i]['Titles']) 
                print(str(i+1)+"-> "+res[0])
                print(df_temp.iloc[i]['Source']+" , "+df_temp.iloc[i]['Date'])
        
                print('\n\n')
            except:
                continue
        
        ind=int(input("Please Provide the index of the story"))
            #print(str(stories_checked.iloc[indices[ind-1]]['link']))
    
            #ind=int(input("Please Provide the index of the story"))
        webbrowser.open(df_temp.iloc[ind-1]['link'])
        time.sleep(3)
        try:
                file_u = open('user_records.json')
                users=json.load(file_u)

                if email not in users.keys():
                    users[email]={}
                
                    users[email]['news']=[df_temp[ind-1]['Source']+df_temp.iloc[ind-1]['Date']+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0]]
                    lab=[z for z in ast.literal_eval(df_temp.iloc[ind-1]['labels'])]
                    users[email]['labels']=lab
                else:
                    users[email]['news'].append(df_temp.iloc[ind-1]['Source']+df_temp.iloc[ind-1]['Date']+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0])
                    lab=[z for z in ast.literal_eval(df_temp.iloc[ind-1]['labels'])]
                    for l in lab:
                        users[email]['labels'].append(l)
                
                with open("user_records.json", "w") as outfile: 
                    json.dump(users, outfile)

                file_s = open('story_records.json')
                stories=json.load(file_s)
                key=df_temp.iloc[ind-1]['Source']+'$'+df_temp.iloc[ind-1]['Date']+'$'+ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0]

                if key not in stories.keys():

                    stories[key]=[email]
                else:
                    stories[key].append(email)

                with open("story_records.json", "w") as outfile: 
                    json.dump(stories, outfile) 
        except:
                a=1
        handle_similar(df,ast.literal_eval(df_temp.iloc[ind-1]['Titles'])[0],email)
         
    else:
        get_popular(email,df)   

In [337]:
from datetime import datetime 
import random
import ast,time
import webbrowser
import json
def surf(email):
    clear()
    date=datetime.today().strftime('%Y-%m-%d')
    
    stories=get_stories()
    
    stories_checked=stories[stories["Date"]==date]
    length=len(stories_checked)
    print()
    indices=[]
    
    for _ in range(8):
        indices.append(random.randint(0,length))
    
    print("LATEST FEED: \n\n")
    
    for i in range(len(indices)-1):
        try:
            #print(indices[i])
            res = ast.literal_eval(stories_checked.iloc[indices[i]]['Titles']) 
            print(str(i+1)+"-> "+res[0])
            print(stories_checked.iloc[indices[i]]['Source']+", Today")
        
            print('\n\n')
        except:
            continue
        
    print('1 for searching, 2 for reading from feed, 3 for popular stories, 4 for customized stories, 5 to quit')
    
    choice=int(input())
    if choice==2:
        
            ind=int(input("Please Provide the index of the story"))
            #print(str(stories_checked.iloc[indices[ind-1]]['link']))
            webbrowser.open(str(stories_checked.iloc[indices[ind-1]]['link']))
            time.sleep(3)
        
            file_u = open('user_records.json')
            users=json.load(file_u)

            if email not in users.keys():
                users[email]={}
                
                users[email]['news']=[stories_checked.iloc[indices[ind-1]]['Source']+stories_checked.iloc[indices[ind-1]]['Date']+ast.literal_eval(stories_checked.iloc[indices[ind-1]]['Titles'])[0]]
                lab=[z for z in ast.literal_eval(stories_checked.iloc[indices[ind-1]]['labels'])]
                users[email]['labels']=lab
            else:
                users[email]['news'].append(stories_checked.iloc[indices[ind-1]]['Source']+stories_checked.iloc[indices[ind-1]]['Date']+ast.literal_eval(stories_checked.iloc[indices[ind-1]]['Titles'])[0])
                lab=[z for z in ast.literal_eval(stories_checked.iloc[indices[ind-1]]['labels'])]
                for l in lab:
                    users[email]['labels'].append(l)
            #json.dump(users,'user_records.json')
            #print(users)
            
            with open("user_records.json", "w") as outfile: 
                json.dump(users, outfile) 

            file_s = open('story_records.json')
            stories=json.load(file_s)
            key=stories_checked.iloc[indices[ind-1]]['Source']+'$'+stories_checked.iloc[indices[ind-1]]['Date']+'$'+ast.literal_eval(stories_checked.iloc[indices[ind-1]]['Titles'])[0]

            if key not in stories.keys():

                stories[key]=[email]
            else:
                stories[key].append(email)
            with open("story_records.json", "w") as outfile: 
                json.dump(stories, outfile) 

            a=1
        #json.dump(stories,'story_records.json')
        
        
            handle_similar(stories_checked,ast.literal_eval(stories_checked.iloc[indices[ind-1]]['Titles'])[0],email)
    elif choice==1:
        search(email,stories)
    
    elif choice==3:
        get_popular(email,stories)
    
    elif choice==4:
        get_customized(email,stories)
        
    elif choice==5:
        print("BYE")
        
        
    
    
    

In [338]:
login()

1 to Signup, 2 to Login
2
Email:XYZ@gmail.com
Pass:1234
Welcome XYZ

LATEST FEED: 


1-> With single-day spike of 75,809 cases and record 1,133 deaths, India's COVID-19 tally nears 43 lakh mark
Times_Now, Today



2->  Sikhs to be counted as a separate ethnic group in 2020 US census for first time
IndiaToday, Today



3->  IMPACT FEATURE Campus placements go virtual as universities adapt
IndiaToday, Today



4-> Delhi BJP Stages Protest, Says AAP Govt Should Release Funds Of Municipal Corporations
Republic TV, Today



5-> Kerala gold smuggling case: Swapna Suresh shifted to ICU after complaining of chest pain
Times_Now, Today



6->  Govt orders clinical study into Covid-19 re-infection after Bengaluru case
IndiaToday, Today



7-> Subramanian Swamy Questions EAM Jaishankar's Scheduled Meeting With Chinese FM In Moscow
Republic TV, Today



1 for searching, 2 for reading from feed, 3 for popular stories, 4 for customized stories, 5 to quit
2
Please Provide the index of the story4


 H

In [342]:
login()

1 to Signup, 2 to Login
2
Email:XYZ@gmail.com
Pass:1234
Welcome XYZ

LATEST FEED: 


1 for searching, 2 for reading from feed, 3 for popular stories, 4 for customized stories, 5 to quit
4


 Top 10 Results 

1-> With single-day spike of 75,809 cases and record 1,133 deaths, India's COVID-19 tally nears 43 lakh mark
Times_Now , 2020-09-08



2-> Rhea Chakraborty admits to Narcotics Control Bureau she was procuring drugs 'but never consumed them'
Times_Now , 2020-09-06



3-> Kerala Trumps States in Literacy Rate, Delhi Emerges 2nd Best; Andhra Ranks Worst
News18 , 2020-09-07



4-> India Didn't Prioritise Mental Health Before Covid-19. Now it's Paying the Price
News18 , 2020-09-07



5-> Sandip Ssingh Shares Chats With Sushant, Sister Mitu Singh: My Silence Broke 20 Years of My Image
News18 , 2020-09-07



6-> Cristiano Ronaldo Forgot to Wear a Face Mask in the Stands, But Proved He's Not a Covidiot
News18 , 2020-09-07



7-> Vodafone-Idea to Make Major Announcement at 11:45AM Today; Am

KeyboardInterrupt: 